In [1]:
devs = pd.read_csv("../data/LIHTC.csv")

In [2]:
barnes = pd.read_csv('../data/barnes.csv')

# Look for overlap between the two tables

In [3]:
#turn the address column of both tables to all uppercase
barnes['Street Address'] = barnes['Street Address'].str.upper()
devs['PROJ_ADD'] = devs['PROJ_ADD'].str.upper()

In [4]:
for i in barnes['Street Address']:
    for j in devs['PROJ_ADD']:
        if i == j:
            print(f'{i} matches {j}')

5636 OLD HICKORY BLVD matches 5636 OLD HICKORY BLVD


potentially, there is only one match

need 
- name
- location
- when it was built

# Create Complete Development Table

In [5]:
#columns that need to be dropped 
columns_to_keep = ["PROJECT","PROJ_ADD","YR_ALLOC", "YR_PIS", "LATITUDE","LONGITUDE"]
devs = devs[columns_to_keep]

### Fix the year column in the devs table

In [6]:
#create new year column to put the good values into 
devs['year'] = 0
for index, row in devs.iterrows():
    #if yr alloc is good, keep it and move on
    if row['YR_ALLOC'] <= 2022:
        devs.loc[index, 'year'] = row['YR_ALLOC']
        print(f"kept {row['YR_ALLOC']} from {row['PROJECT']}")
    
    #if yr_alloc is no good, but PIS is, then keep PIS
    elif row['YR_PIS'] <= 2022:
        devs.loc[index, 'year'] = row['YR_PIS']
        print(f"kept {row['YR_PIS']} from {row['PROJECT']}")

    #if both are bad, then delete the row from the dataframe
    else:
        print(f"dropped {row['PROJECT']} because years {row['YR_ALLOC']} and {row['YR_PIS']} are not valid")
        devs = devs.drop(index)

kept 2013 from CUMBERLAND POINTE APTS
kept 2013 from HICKORY LAKE APTS
kept 2011 from SEVEN HUNDRED FIVE WOODLAND
kept 2015 from LEVY PLACE
dropped THE PADDOCK AT GRANDVIEW because years 9999 and 9999 are not valid
kept 1987 from 117 W TRINITY LN
kept 1987 from FIFTH AVE SFD
kept 1987 from FIFTH STREET SFD
kept 1987 from 209 A&B PRINCE AVE
kept 1987 from 909 C&D THOMAS AVE
kept 1987 from THOMAS A
kept 1987 from 11TH S ST DUPLEX
kept 1988 from TRI-COUNTY DUPLEX
kept 1988 from AVONDALE SFD
kept 1988 from MERIDIAN APT
kept 1988 from TRI COUNTY DUPLEX
kept 1988 from 14TH AVE N DUPLEX
kept 1988 from 22ND AVE N DUPLEX
kept 1988 from 5TH AVE N DUPLEX
kept 1988 from SOUTHSIDE II
kept 1988 from SOUTHSIDE I
kept 1988 from SOUTHSIDE II - NUBELL ST
kept 1988 from SOUTHSIDE I
kept 1988 from SOUTHSIDE I
kept 1988 from SOUTHSIDE I
kept 1988 from SOUTHSIDE I
kept 1988 from CEPHAS SFD
kept 1988 from SOUTHSIDE II - BERRY ST
kept 1988 from BERRY ST APTS (NASHVILLE)
kept 1988 from SOUTHSIDE I
kept 1988 fr

##### delete the old year columns from the dev table

In [7]:
devs = devs.drop(columns = ['YR_ALLOC', 'YR_PIS'])

### Clean up the barenes table 

In [8]:
#clean up the barnes table now
columns_to_keep = ["Development Name", "Street Address", "Barnes Year", "lat","lng"]
barnes = barnes[columns_to_keep]

#### Rename all the columns to match

In [9]:
new_names = {'PROJECT':'name', 'PROJ_ADD':'address','LATITUDE':'lat', 'LONGITUDE':'lng'}
devs = devs.rename(columns = new_names)

In [10]:
new_names =  {'Development Name':'name', 'Street Address':'address', 'Barnes Year':'year'}
barnes = barnes.rename(columns  = new_names)

In [11]:
#drop the overlapping development, remember to only run this cell once
barnes = barnes.drop(3)

In [12]:
all_devs  = pd.concat([devs, barnes])

In [13]:
all_devs

,name,address,lat,lng,year
0,CUMBERLAND POINTE APTS,2400 BUENA VISTA PIKE,36.199924,-86.825104,2013
1,HICKORY LAKE APTS,3940 APACHE TRAIL,36.067261,-86.691025,2013
2,SEVEN HUNDRED FIVE WOODLAND,705 WOODLAND ST,36.173809,-86.760063,2011
3,LEVY PLACE,303 FOSTER ST.,36.179005,-86.767433,2015
5,117 W TRINITY LN,117 W TRINITY LN,36.206188,-86.770447,1987
...,...,...,...,...,...
4,NaN,2069 WHITNEY AVENUE,36.129724,-86.751635,2017
5,Gibson Creek,300 EAST WEBSTER STREET,36.248686,-86.711886,2017
6,Restoration on Lindsley,35 LINDSLEY AVENUE,36.153509,-86.765865,2019
7,NaN,3600 CLARKSVILLE PIKE,36.197128,-86.835643,2019


**development table saved as all_devs**

# Filter table to only show developments that came into existence after/during 2010

In [14]:
recent_devs = all_devs.loc[all_devs.year >= 2010]

In [15]:
recent_devs = recent_devs.sort_values(by = 'year').reset_index(drop=True)

In [16]:
recent_devs

,name,address,lat,lng,year
0,RYMAN LOFTS AT ROLLING MILL HILL,100 MIDDLETON CIR,36.046509,-86.875031,2010
1,SEVEN HUNDRED FIVE WOODLAND,705 WOODLAND ST,36.173809,-86.760063,2011
2,SKYLINE VILLAGE APTS,3225 CREEKWOOD DR,36.230930,-86.771362,2012
3,WILLOW POINTE,120 HICKORY HOLLOW TER,36.158840,-86.643936,2012
4,TERRACE PARK TOWNHOMES PHASE II,3110 ELM HILL PIKE,36.142052,-86.639954,2012
5,PARK AT RICHARDS ROAD APTS,850 RICHARDS RD,36.068661,-86.674683,2012
6,HALLMARK STATION,3970 CLARKSVILLE PIKE,36.210262,-86.838890,2013
7,TOWNHOMES OF NASHBORO VILLAGE,1067 NASHBORO BLVD,36.089867,-86.640373,2013
8,THE RETREAT AT DRY CREEK FARMS PHASE II,1816 ROBERT CARTWRIGHT DR,36.346298,-86.739403,2013
9,CUMBERLAND POINTE APTS,2400 BUENA VISTA PIKE,36.199924,-86.825104,2013


In [17]:
recent_devs.shape

(27, 5)

In [18]:
recent_devs.to_csv('../created_tables/developments.csv', index=False)